# Cuttlefish3: LangGraph Multi-Agent RAG System

A sophisticated multi-agent system for intelligent JIRA ticket retrieval using LangGraph.

## System Architecture:
- **Supervisor Agent**: Intelligent query routing (GPT-4o)
- **BM25 Agent**: Keyword-based search
- **ContextualCompression Agent**: Fast semantic retrieval with reranking  
- **Ensemble Agent**: Comprehensive multi-method retrieval
- **ResponseWriter Agent**: Contextual response generation (GPT-4o)

## Routing Logic:
- **Keyword queries** → BM25 Agent
- **user_can_wait=True** → Ensemble Agent (~47s)
- **production_incident=True** → ContextualCompression Agent (urgent ~21s)
- **Default** → ContextualCompression Agent

## Phase 1: Setup & Infrastructure

### Cell 1: Dependencies & Configuration

In [ ]:
# Install required packages
!pip install -q langgraph langsmith langchain-openai langchain-community
!pip install -q qdrant-client langchain-qdrant rank-bm25 langchain-cohere
!pip install -q flask flask-cors python-dotenv
!pip install -q "cohere>=5.12.0,<5.13.0" langchain-cohere==0.4.4

In [ ]:
import os
import getpass
from typing import Dict, List, Any, Optional, TypedDict, Annotated
from uuid import uuid4
import json
from datetime import datetime

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# OpenAI and embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Qdrant and retrievers
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# Flask for API
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ All dependencies imported successfully!")

In [ ]:
# Configuration and API Keys Setup

# Model Configuration
REASONING_MODEL = "gpt-4o"  # For Supervisor and ResponseWriter agents
TASK_MODEL = "gpt-4o-mini"  # For RAG agents
EMBEDDING_MODEL = "text-embedding-3-small"

# Qdrant Configuration
QDRANT_URL = os.environ.get('QDRANT_URL')
QDRANT_API_KEY = os.environ.get('QDRANT_API_KEY')
QDRANT_COLLECTION = os.environ.get('QDRANT_COLLECTION', 'jira_issues_semantic')

# API Keys
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
    
if "COHERE_API_KEY" not in os.environ:
    os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API Key: ")

# LangSmith Configuration for debugging and monitoring
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = f"Cuttlefish3-MultiAgent-{uuid4().hex[0:8]}"

# Validate required environment variables
required_vars = ['QDRANT_URL', 'QDRANT_API_KEY'] if QDRANT_URL else []
missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    print(f"⚠️  Missing Qdrant configuration: {', '.join(missing_vars)}")
    print("Will use in-memory vectorstore for testing")
    USE_REMOTE_QDRANT = False
else:
    USE_REMOTE_QDRANT = True
    print(f"✅ Using remote Qdrant: {QDRANT_URL}")

print(f"✅ Configuration complete!")
print(f"   Reasoning Model: {REASONING_MODEL}")
print(f"   Task Model: {TASK_MODEL}")
print(f"   Embedding Model: {EMBEDDING_MODEL}")
print(f"   LangSmith Project: {os.environ['LANGCHAIN_PROJECT']}")
print(f"   Qdrant Collection: {QDRANT_COLLECTION}")

### Cell 2: Qdrant Connection & Data Setup

In [ ]:
# Initialize OpenAI models and embeddings
print("🔧 Initializing models...")

# Reasoning models for complex decision making
supervisor_llm = ChatOpenAI(model=REASONING_MODEL, temperature=0.1)
response_writer_llm = ChatOpenAI(model=REASONING_MODEL, temperature=0.2)

# Task models for straightforward RAG operations
rag_llm = ChatOpenAI(model=TASK_MODEL, temperature=0.1)

# Embeddings for vector operations
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

print("✅ Models initialized successfully!")

In [ ]:
# Setup Qdrant connection and load JIRA data
print("🔌 Setting up Qdrant connection...")

try:
    if USE_REMOTE_QDRANT:
        # Connect to remote Qdrant instance
        qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
        
        # Test connection and get collection info
        collection_info = qdrant_client.get_collection(QDRANT_COLLECTION)
        point_count = collection_info.points_count
        
        # Initialize vectorstore
        vectorstore = QdrantVectorStore(
            client=qdrant_client,
            collection_name=QDRANT_COLLECTION,
            embedding=embeddings
        )
        
        print(f"✅ Connected to remote Qdrant: {QDRANT_COLLECTION}")
        print(f"   Collection points: {point_count:,}")
        print(f"   Vector size: {collection_info.config.params.vectors.size}")
        
    else:
        # Fallback: Use sample data for testing without remote Qdrant
        print("📝 Creating sample JIRA data for testing...")
        
        from langchain_core.documents import Document
        
        # Sample JIRA documents for testing
        sample_docs = [
            Document(
                page_content="Title: Memory leak in XML parser\n\nDescription: Application crashes after processing multiple XML files due to memory not being freed properly in Xerces-C++ library.",
                metadata={"key": "HBASE-001", "project": "HBASE", "priority": "Critical", "type": "Bug"}
            ),
            Document(
                page_content="Title: ClassCastException in SAXParserFactory\n\nDescription: Getting ClassCastException when trying to create SAX parser factory in multi-threaded environment.",
                metadata={"key": "FLEX-002", "project": "FLEX", "priority": "Major", "type": "Bug"}
            ),
            Document(
                page_content="Title: Maven archetype generation fails\n\nDescription: Maven archetype:generate command fails with dependency resolution errors in offline mode.",
                metadata={"key": "SPR-003", "project": "SPR", "priority": "Minor", "type": "Bug"}
            ),
            Document(
                page_content="Title: ZooKeeper quota exceeded\n\nDescription: ZooKeeper client throws quota exceeded exception when creating more than 1000 znodes.",
                metadata={"key": "HBASE-004", "project": "HBASE", "priority": "Major", "type": "Bug"}
            ),
            Document(
                page_content="Title: Hibernate lazy loading issue\n\nDescription: LazyInitializationException occurs when accessing lazy-loaded collections outside of session scope.",
                metadata={"key": "JBIDE-005", "project": "JBIDE", "priority": "Critical", "type": "Bug"}
            )
        ]
        
        # Create in-memory vectorstore
        from langchain_community.vectorstores import Qdrant
        
        vectorstore = Qdrant.from_documents(
            sample_docs,
            embeddings,
            location=":memory:",
            collection_name="jira_test"
        )
        
        print(f"✅ Created sample vectorstore with {len(sample_docs)} documents")
        
except Exception as e:
    print(f"❌ Error setting up Qdrant: {e}")
    print("Creating minimal test setup...")
    
    # Minimal fallback
    from langchain_core.documents import Document
    from langchain_community.vectorstores import Qdrant
    
    test_doc = Document(
        page_content="Title: Test JIRA issue\n\nDescription: This is a test document for the multi-agent system.",
        metadata={"key": "TEST-001", "project": "TEST", "priority": "Low", "type": "Task"}
    )
    
    vectorstore = Qdrant.from_documents(
        [test_doc],
        embeddings,
        location=":memory:",
        collection_name="test"
    )
    
    print("✅ Minimal test setup complete")

In [ ]:
---\n## ✅ Phase 4 Complete: Flask API Implementation\n\n**Implemented:**\n- ✅ **Flask App Setup**: CORS configuration following cuttlefish2-main.py pattern\n- ✅ **Main API Endpoint**: `/multiagent-rag` - Full multi-agent processing\n- ✅ **Health Check**: `/health` - Service status and agent information\n- ✅ **Debug Endpoint**: `/debug/routing` - Test routing decisions without full processing\n- ✅ **Interactive Test Interface**: Beautiful HTML UI with sample queries\n- ✅ **Error Handling**: Comprehensive error handling and logging\n- ✅ **API Key Support**: Optional OpenAI API key per request\n\n**API Features:**\n- 🎯 **Request Model**: `{query, user_can_wait, production_incident, openai_api_key?}`\n- 📊 **Response Model**: `{answer, context[], metadata}` - Compatible with cuttlefish2\n- 🔧 **CORS Support**: Ready for frontend integration\n- 🧪 **Testing UI**: Interactive interface with sample queries\n- 📈 **Rich Metadata**: Routing decisions, performance metrics, agent information\n\n**Usage:**\n```python\n# Start the server\nrun_server()  # Default: localhost:5000\n\n# Or with custom settings\nrun_server(host='0.0.0.0', port=8080)\n```\n\n**Endpoints:**\n- `GET /` - Interactive testing interface\n- `GET /health` - System health and agent status\n- `POST /multiagent-rag` - Main multi-agent RAG endpoint\n- `POST /debug/routing` - Routing decision testing\n\n**Ready for Phase 5:** Documentation & final validation!"

In [ ]:
# Server Launch Function\ndef run_server(host='127.0.0.1', port=5000, debug=True):\n    \"\"\"Launch the Flask server.\"\"\"\n    print(f\"\\n🚀 Starting Cuttlefish3 Multi-Agent RAG Server...\")\n    print(f\"   Server URL: http://{host}:{port}\")\n    print(f\"   Health Check: http://{host}:{port}/health\")\n    print(f\"   Main API: http://{host}:{port}/multiagent-rag\")\n    print(f\"   Debug API: http://{host}:{port}/debug/routing\")\n    print(f\"   Test Interface: http://{host}:{port}/\")\n    print(f\"   LangSmith Project: {os.environ['LANGCHAIN_PROJECT']}\")\n    print(\"-\" * 60)\n    \n    try:\n        app.run(host=host, port=port, debug=debug)\n    except KeyboardInterrupt:\n        print(\"\\n👋 Server stopped\")\n    except Exception as e:\n        print(f\"❌ Server error: {e}\")\n\nprint(\"✅ Server launch function ready\")\nprint(\"\\n🎯 To start the server, run: run_server()\")\nprint(\"   Or with custom settings: run_server(host='0.0.0.0', port=8080)\")"

In [ ]:
# HTML Testing Interface\nTEST_INTERFACE_HTML = \"\"\"\n<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n    <meta charset=\"UTF-8\">\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">\n    <title>Cuttlefish3 Multi-Agent RAG System</title>\n    <style>\n        body { font-family: Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px; }\n        .container { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }\n        .panel { border: 1px solid #ddd; padding: 20px; border-radius: 8px; }\n        .input-group { margin-bottom: 15px; }\n        label { display: block; margin-bottom: 5px; font-weight: bold; }\n        input[type=\"text\"], textarea { width: 100%; padding: 8px; border: 1px solid #ccc; border-radius: 4px; }\n        textarea { height: 100px; resize: vertical; }\n        .checkbox-group { display: flex; gap: 20px; margin: 10px 0; }\n        .checkbox-group label { font-weight: normal; }\n        button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 4px; cursor: pointer; }\n        button:hover { background: #0056b3; }\n        button:disabled { background: #ccc; cursor: not-allowed; }\n        .response { background: #f8f9fa; padding: 15px; border-radius: 4px; margin-top: 15px; }\n        .metadata { font-size: 0.9em; color: #666; margin-top: 10px; }\n        .error { background: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }\n        .success { background: #d4edda; color: #155724; border: 1px solid #c3e6cb; }\n        .loading { color: #007bff; }\n        .urgent { background: #fff3cd; border: 1px solid #ffeaa7; }\n        .comprehensive { background: #e7f3ff; border: 1px solid #b3d9ff; }\n        .tickets { margin-top: 10px; }\n        .ticket { background: white; padding: 8px; margin: 5px 0; border-left: 3px solid #007bff; }\n    </style>\n</head>\n<body>\n    <h1>🐙 Cuttlefish3 Multi-Agent RAG System</h1>\n    <p>Intelligent JIRA ticket retrieval using GPT-4o-powered multi-agent architecture</p>\n    \n    <div class=\"container\">\n        <div class=\"panel\">\n            <h3>Query Interface</h3>\n            <form id=\"ragForm\">\n                <div class=\"input-group\">\n                    <label for=\"query\">JIRA Query:</label>\n                    <textarea id=\"query\" placeholder=\"Ask about JIRA tickets, bugs, or technical issues...\">How to fix memory leaks in XML parser?</textarea>\n                </div>\n                \n                <div class=\"checkbox-group\">\n                    <label><input type=\"checkbox\" id=\"userCanWait\"> User can wait (comprehensive search)</label>\n                    <label><input type=\"checkbox\" id=\"productionIncident\"> Production incident (urgent)</label>\n                </div>\n                \n                <div class=\"input-group\">\n                    <label for=\"apiKey\">OpenAI API Key (optional):</label>\n                    <input type=\"password\" id=\"apiKey\" placeholder=\"sk-...\">\n                </div>\n                \n                <button type=\"submit\" id=\"submitBtn\">🚀 Process Query</button>\n                <button type=\"button\" id=\"debugBtn\" style=\"margin-left: 10px; background: #6c757d;\">🔍 Debug Routing</button>\n            </form>\n        </div>\n        \n        <div class=\"panel\">\n            <h3>Sample Queries</h3>\n            <div style=\"margin-bottom: 10px;\">\n                <button type=\"button\" onclick=\"setQuery('HBASE-123')\">🔍 Specific Ticket</button>\n                <button type=\"button\" onclick=\"setQuery('Production system down with ClassCastException', false, true)\">🚨 Production Issue</button>\n                <button type=\"button\" onclick=\"setQuery('What are common Maven build failures?', true, false)\">📚 Research Query</button>\n            </div>\n            <p style=\"font-size: 0.9em; color: #666;\">Try different query types to see intelligent routing in action!</p>\n        </div>\n    </div>\n    \n    <div class=\"panel\" style=\"margin-top: 20px;\">\n        <h3>Response</h3>\n        <div id=\"response\">Submit a query to see the response...</div>\n    </div>\n\n    <script>\n        function setQuery(query, canWait = false, incident = false) {\n            document.getElementById('query').value = query;\n            document.getElementById('userCanWait').checked = canWait;\n            document.getElementById('productionIncident').checked = incident;\n        }\n        \n        async function makeRequest(endpoint, data) {\n            const response = await fetch(endpoint, {\n                method: 'POST',\n                headers: { 'Content-Type': 'application/json' },\n                body: JSON.stringify(data)\n            });\n            return await response.json();\n        }\n        \n        function formatResponse(data, isDebug = false) {\n            const responseDiv = document.getElementById('response');\n            \n            if (data.error) {\n                responseDiv.innerHTML = `<div class=\"response error\"><strong>Error:</strong> ${data.error}</div>`;\n                return;\n            }\n            \n            if (isDebug) {\n                responseDiv.innerHTML = `\n                    <div class=\"response\">\n                        <h4>🧠 Routing Decision</h4>\n                        <p><strong>Agent:</strong> ${data.routing_decision}</p>\n                        <p><strong>Reasoning:</strong> ${data.routing_reasoning}</p>\n                        <div class=\"metadata\">\n                            Query: \"${data.query}\" | Can Wait: ${data.user_can_wait} | Incident: ${data.production_incident}\n                        </div>\n                    </div>\n                `;\n                return;\n            }\n            \n            const urgentClass = data.metadata?.production_incident ? 'urgent' : '';\n            const comprehensiveClass = data.metadata?.routing_decision === 'Ensemble' ? 'comprehensive' : '';\n            \n            let ticketsHtml = '';\n            if (data.context && data.context.length > 0) {\n                ticketsHtml = `\n                    <div class=\"tickets\">\n                        <h5>📋 Relevant Tickets (${data.context.length}):</h5>\n                        ${data.context.map(ticket => `\n                            <div class=\"ticket\">\n                                <strong>${ticket.key}</strong>: ${ticket.title}\n                            </div>\n                        `).join('')}\n                    </div>\n                `;\n            }\n            \n            responseDiv.innerHTML = `\n                <div class=\"response success ${urgentClass} ${comprehensiveClass}\">\n                    <h4>💬 Answer</h4>\n                    <p>${data.answer}</p>\n                    ${ticketsHtml}\n                    <div class=\"metadata\">\n                        <strong>Routing:</strong> ${data.metadata?.routing_decision} (${data.metadata?.routing_reasoning}) |\n                        <strong>Method:</strong> ${data.metadata?.retrieval_method} |\n                        <strong>Time:</strong> ${data.metadata?.processing_time?.toFixed(2)}s |\n                        <strong>Tickets:</strong> ${data.metadata?.num_tickets_found || 0}\n                    </div>\n                </div>\n            `;\n        }\n        \n        document.getElementById('ragForm').addEventListener('submit', async (e) => {\n            e.preventDefault();\n            \n            const submitBtn = document.getElementById('submitBtn');\n            const responseDiv = document.getElementById('response');\n            \n            submitBtn.disabled = true;\n            responseDiv.innerHTML = '<div class=\"response loading\">🔄 Processing query through multi-agent system...</div>';\n            \n            const data = {\n                query: document.getElementById('query').value,\n                user_can_wait: document.getElementById('userCanWait').checked,\n                production_incident: document.getElementById('productionIncident').checked,\n                openai_api_key: document.getElementById('apiKey').value || undefined\n            };\n            \n            try {\n                const result = await makeRequest('/multiagent-rag', data);\n                formatResponse(result);\n            } catch (error) {\n                responseDiv.innerHTML = `<div class=\"response error\"><strong>Network Error:</strong> ${error.message}</div>`;\n            } finally {\n                submitBtn.disabled = false;\n            }\n        });\n        \n        document.getElementById('debugBtn').addEventListener('click', async () => {\n            const responseDiv = document.getElementById('response');\n            responseDiv.innerHTML = '<div class=\"response loading\">🔍 Testing routing decision...</div>';\n            \n            const data = {\n                query: document.getElementById('query').value,\n                user_can_wait: document.getElementById('userCanWait').checked,\n                production_incident: document.getElementById('productionIncident').checked\n            };\n            \n            try {\n                const result = await makeRequest('/debug/routing', data);\n                formatResponse(result, true);\n            } catch (error) {\n                responseDiv.innerHTML = `<div class=\"response error\"><strong>Debug Error:</strong> ${error.message}</div>`;\n            }\n        });\n    </script>\n</body>\n</html>\n\"\"\"\n\n@app.route('/', methods=['GET'])\ndef test_interface():\n    \"\"\"Serve the testing interface.\"\"\"\n    return TEST_INTERFACE_HTML\n\nprint(\"✅ Test interface defined\")"

### Cell 14: Testing Interface & Server Launch

In [ ]:
# API Endpoints\n\n@app.route('/health', methods=['GET'])\ndef health_check():\n    \"\"\"Health check endpoint.\"\"\"\n    return jsonify({\n        'status': 'healthy',\n        'service': 'Cuttlefish3 Multi-Agent RAG',\n        'version': '1.0.0',\n        'timestamp': datetime.now().isoformat(),\n        'agents': {\n            'supervisor': 'GPT-4o',\n            'response_writer': 'GPT-4o', \n            'bm25': 'operational',\n            'contextual_compression': 'operational',\n            'ensemble': 'operational'\n        }\n    })\n\n@app.route('/multiagent-rag', methods=['POST'])\ndef multiagent_rag_endpoint():\n    \"\"\"Multi-agent RAG endpoint - main API for intelligent JIRA ticket retrieval.\"\"\"\n    try:\n        # Parse request\n        data = request.get_json()\n        if not data:\n            return jsonify({'error': 'No JSON data provided'}), 400\n        \n        # Validate required fields\n        query = data.get('query', '').strip()\n        if not query:\n            return jsonify({'error': 'Query is required'}), 400\n        \n        user_can_wait = data.get('user_can_wait', False)\n        production_incident = data.get('production_incident', False)\n        openai_api_key = data.get('openai_api_key')\n        \n        # Temporarily update OpenAI API key if provided\n        original_key = None\n        if openai_api_key:\n            original_key = os.environ.get('OPENAI_API_KEY')\n            os.environ['OPENAI_API_KEY'] = openai_api_key\n        \n        try:\n            # Process query through multi-agent system\n            result = process_query(\n                query=query,\n                user_can_wait=user_can_wait,\n                production_incident=production_incident\n            )\n            \n            # Return successful response\n            return jsonify(result)\n            \n        except Exception as processing_error:\n            print(f\"❌ Processing error: {processing_error}\")\n            return jsonify({\n                'error': f'Processing failed: {str(processing_error)}',\n                'query': query,\n                'timestamp': datetime.now().isoformat()\n            }), 500\n            \n        finally:\n            # Restore original API key\n            if original_key:\n                os.environ['OPENAI_API_KEY'] = original_key\n    \n    except Exception as e:\n        print(f\"❌ Endpoint error: {e}\")\n        return jsonify({\n            'error': f'Request failed: {str(e)}',\n            'timestamp': datetime.now().isoformat()\n        }), 500\n\n@app.route('/debug/routing', methods=['POST'])\ndef debug_routing():\n    \"\"\"Debug endpoint to test routing decisions without full processing.\"\"\"\n    try:\n        data = request.get_json()\n        if not data:\n            return jsonify({'error': 'No JSON data provided'}), 400\n        \n        query = data.get('query', '').strip()\n        if not query:\n            return jsonify({'error': 'Query is required'}), 400\n        \n        user_can_wait = data.get('user_can_wait', False)\n        production_incident = data.get('production_incident', False)\n        \n        # Get routing decision from supervisor\n        routing_result = supervisor_agent.route_query(query, user_can_wait, production_incident)\n        \n        return jsonify({\n            'query': query,\n            'user_can_wait': user_can_wait,\n            'production_incident': production_incident,\n            'routing_decision': routing_result['agent'],\n            'routing_reasoning': routing_result['reasoning'],\n            'timestamp': datetime.now().isoformat()\n        })\n        \n    except Exception as e:\n        return jsonify({\n            'error': f'Routing debug failed: {str(e)}',\n            'timestamp': datetime.now().isoformat()\n        }), 500\n\nprint(\"✅ API endpoints defined\")"

### Cell 13: API Endpoints Implementation

In [ ]:
# Flask API Implementation for Cuttlefish3 Multi-Agent System\nfrom dataclasses import dataclass\nfrom typing import Optional\nimport json\n\n# Request and Response Models\n@dataclass\nclass MultiAgentRequest:\n    \"\"\"Request model for multi-agent RAG endpoint.\"\"\"\n    query: str\n    user_can_wait: bool = False\n    production_incident: bool = False\n    openai_api_key: Optional[str] = None\n\n@dataclass \nclass TicketContext:\n    \"\"\"JIRA ticket context model.\"\"\"\n    key: str\n    title: str\n    score: float\n    payload: dict\n\n@dataclass\nclass MultiAgentResponse:\n    \"\"\"Response model for multi-agent RAG endpoint.\"\"\"\n    answer: str\n    context: List[TicketContext]\n    metadata: dict\n\n# Initialize Flask app\napp = Flask(__name__)\n\n# CORS configuration - following cuttlefish2-main.py pattern\nCORS(app, \n     origins=[\"*\"],  # For development - restrict in production\n     allow_credentials=True,\n     allow_methods=[\"GET\", \"POST\", \"OPTIONS\"],\n     allow_headers=[\"*\"]\n)\n\nprint(\"✅ Flask app initialized with CORS configuration\")"

### Cell 12: Flask API Setup & Request Models

## Phase 4: Flask API Implementation

---\n## ✅ Phase 3 Complete: LangGraph Workflow Orchestration\n\n**Implemented:**\n- ✅ **Agent Node Functions**: Wrapped all agents for LangGraph integration\n- ✅ **Conditional Routing**: Intelligent routing based on Supervisor decisions  \n- ✅ **Graph Construction**: Complete workflow with proper edge connections\n- ✅ **Multi-Agent Interface**: Clean API for processing queries\n- ✅ **Integration Testing**: Validated all routing scenarios\n\n**Workflow Architecture:**\n```\nSupervisor (GPT-4o) → [BM25 | ContextualCompression | Ensemble] → ResponseWriter (GPT-4o) → End\n```\n\n**Routing Logic Verified:**\n- 🔍 **Keyword queries** → BM25 Agent (fast)\n- ⚡ **Production incidents** → ContextualCompression Agent (urgent mode)\n- 🔗 **Comprehensive queries** → Ensemble Agent (thorough)\n- 🛡️ **Default/fallback** → ContextualCompression Agent\n\n**Key Features:**\n- 🧠 **GPT-4o reasoning** for routing and response generation\n- 📊 **Full LangSmith tracing** for debugging and monitoring\n- 🔄 **Error handling** with graceful fallbacks\n- ⏱️ **Performance tracking** across all agents\n- 🎯 **Contextual awareness** for production incidents\n\n**Ready for Phase 4:** Flask API implementation!"

In [ ]:
# Test the Multi-Agent System\nprint(\"🧪 Testing Multi-Agent System with different scenarios...\\n\")\n\n# Test Case 1: BM25 routing (keyword query)\ntest_result_1 = process_query(\n    query=\"How to fix memory leaks in XML parser?\",\n    user_can_wait=False,\n    production_incident=False\n)\n\nprint(f\"\\n📊 Test 1 Results:\")\nprint(f\"   Routing Decision: {test_result_1['metadata']['routing_decision']}\")\nprint(f\"   Processing Time: {test_result_1['metadata']['processing_time']:.2f}s\")\nprint(f\"   Answer: {test_result_1['answer'][:100]}...\")\nprint(f\"   Tickets Found: {test_result_1['metadata']['num_tickets_found']}\")\n\nprint(\"\\n\" + \"=\"*80)\n\n# Test Case 2: Production incident (urgent)\ntest_result_2 = process_query(\n    query=\"Production system is down with ClassCastException\",\n    user_can_wait=False,\n    production_incident=True\n)\n\nprint(f\"\\n📊 Test 2 Results (Production Incident):\")\nprint(f\"   Routing Decision: {test_result_2['metadata']['routing_decision']}\")\nprint(f\"   Processing Time: {test_result_2['metadata']['processing_time']:.2f}s\")\nprint(f\"   Answer: {test_result_2['answer'][:100]}...\")\nprint(f\"   Tickets Found: {test_result_2['metadata']['num_tickets_found']}\")\n\nprint(\"\\n\" + \"=\"*80)\n\n# Test Case 3: Comprehensive search (user can wait)\ntest_result_3 = process_query(\n    query=\"What are common causes of Maven archetype generation failures?\",\n    user_can_wait=True,\n    production_incident=False\n)\n\nprint(f\"\\n📊 Test 3 Results (Comprehensive):\")\nprint(f\"   Routing Decision: {test_result_3['metadata']['routing_decision']}\")\nprint(f\"   Processing Time: {test_result_3['metadata']['processing_time']:.2f}s\")\nprint(f\"   Answer: {test_result_3['answer'][:100]}...\")\nprint(f\"   Tickets Found: {test_result_3['metadata']['num_tickets_found']}\")\n\nprint(\"\\n🎉 Multi-Agent System testing complete!\")"

In [ ]:
# Multi-Agent System Interface\n\ndef process_query(query: str, user_can_wait: bool = False, production_incident: bool = False) -> Dict[str, Any]:\n    \"\"\"\n    Main interface for the multi-agent RAG system.\n    \n    Args:\n        query (str): User query\n        user_can_wait (bool): Whether user can wait for comprehensive results\n        production_incident (bool): Whether this is a production incident (urgent)\n    \n    Returns:\n        Dict containing the response and metadata\n    \"\"\"\n    \n    # Initialize state\n    initial_state = {\n        'query': query,\n        'user_can_wait': user_can_wait,\n        'production_incident': production_incident,\n        'routing_decision': None,\n        'routing_reasoning': None,\n        'retrieved_contexts': [],\n        'retrieval_method': None,\n        'retrieval_metadata': {},\n        'final_answer': None,\n        'relevant_tickets': [],\n        'messages': [HumanMessage(content=query)],\n        'timestamp': datetime.now().isoformat(),\n        'processing_time': None\n    }\n    \n    start_time = datetime.now()\n    \n    try:\n        print(f\"\\n🚀 Processing query: '{query}'\")\n        print(f\"   Settings: user_can_wait={user_can_wait}, production_incident={production_incident}\")\n        print(\"-\" * 80)\n        \n        # Run the multi-agent workflow\n        final_state = multi_agent_rag.invoke(initial_state)\n        \n        # Calculate total processing time\n        total_processing_time = measure_performance(start_time)\n        final_state['processing_time'] = total_processing_time\n        \n        print(\"-\" * 80)\n        print(f\"✅ Query processing completed in {total_processing_time:.2f}s\")\n        \n        # Format response\n        response = {\n            'answer': final_state.get('final_answer', 'No answer generated'),\n            'context': [\n                {\n                    'key': ticket.get('key', ''),\n                    'title': ticket.get('title', ''),\n                    'score': 1.0,  # Default score\n                    'payload': ticket\n                }\n                for ticket in final_state.get('relevant_tickets', [])\n            ],\n            'metadata': {\n                'routing_decision': final_state.get('routing_decision'),\n                'routing_reasoning': final_state.get('routing_reasoning'),\n                'retrieval_method': final_state.get('retrieval_method'),\n                'retrieval_metadata': final_state.get('retrieval_metadata', {}),\n                'processing_time': total_processing_time,\n                'timestamp': final_state.get('timestamp'),\n                'num_tickets_found': len(final_state.get('relevant_tickets', [])),\n                'production_incident': production_incident\n            }\n        }\n        \n        return response\n        \n    except Exception as e:\n        error_time = measure_performance(start_time)\n        print(f\"❌ Error processing query: {e}\")\n        \n        # Return error response\n        return {\n            'answer': f\"Error processing query: {str(e)}\",\n            'context': [],\n            'metadata': {\n                'error': str(e),\n                'processing_time': error_time,\n                'timestamp': datetime.now().isoformat(),\n                'production_incident': production_incident\n            }\n        }\n\nprint(\"✅ Multi-Agent System interface ready\")"

### Cell 11: Multi-Agent System Interface & Testing

In [ ]:
# Router Function for Conditional Routing\n\ndef route_to_agent(state: AgentState) -> str:\n    \"\"\"\n    Route to the appropriate retrieval agent based on supervisor decision.\n    Returns the name of the next node to execute.\n    \"\"\"\n    routing_decision = state.get('routing_decision', 'ContextualCompression')\n    \n    # Map supervisor decisions to node names\n    route_mapping = {\n        'BM25': 'bm25_agent',\n        'ContextualCompression': 'contextual_compression_agent', \n        'Ensemble': 'ensemble_agent'\n    }\n    \n    next_node = route_mapping.get(routing_decision, 'contextual_compression_agent')\n    print(f\"🔀 Routing to: {next_node}\")\n    \n    return next_node\n\n# Create the LangGraph workflow\nfrom langgraph.graph import StateGraph, END\n\nprint(\"🏗️  Building LangGraph workflow...\")\n\n# Initialize the graph\nworkflow = StateGraph(AgentState)\n\n# Add nodes\nworkflow.add_node(\"supervisor\", supervisor_node)\nworkflow.add_node(\"bm25_agent\", bm25_node)\nworkflow.add_node(\"contextual_compression_agent\", contextual_compression_node)\nworkflow.add_node(\"ensemble_agent\", ensemble_node)\nworkflow.add_node(\"response_writer\", response_writer_node)\n\n# Set entry point\nworkflow.set_entry_point(\"supervisor\")\n\n# Add conditional routing from supervisor to retrieval agents\nworkflow.add_conditional_edges(\n    \"supervisor\",\n    route_to_agent,\n    {\n        \"bm25_agent\": \"bm25_agent\",\n        \"contextual_compression_agent\": \"contextual_compression_agent\",\n        \"ensemble_agent\": \"ensemble_agent\"\n    }\n)\n\n# Add edges from all retrieval agents to response writer\nworkflow.add_edge(\"bm25_agent\", \"response_writer\")\nworkflow.add_edge(\"contextual_compression_agent\", \"response_writer\")\nworkflow.add_edge(\"ensemble_agent\", \"response_writer\")\n\n# Add edge from response writer to end\nworkflow.add_edge(\"response_writer\", END)\n\n# Compile the graph\nmulti_agent_rag = workflow.compile()\n\nprint(\"✅ LangGraph workflow compiled successfully!\")\nprint(\"   Workflow: Supervisor → [BM25|ContextualCompression|Ensemble] → ResponseWriter → End\")"

### Cell 10: Router Function & Graph Construction

In [ ]:
# Agent Node Functions for LangGraph\n# These functions wrap our agent classes for LangGraph integration\n\ndef supervisor_node(state: AgentState) -> AgentState:\n    \"\"\"Supervisor node for intelligent query routing.\"\"\"\n    return supervisor_agent.process(state)\n\ndef bm25_node(state: AgentState) -> AgentState:\n    \"\"\"BM25 retrieval node.\"\"\"\n    return bm25_agent.process(state)\n\ndef contextual_compression_node(state: AgentState) -> AgentState:\n    \"\"\"ContextualCompression retrieval node.\"\"\"\n    return contextual_compression_agent.process(state)\n\ndef ensemble_node(state: AgentState) -> AgentState:\n    \"\"\"Ensemble retrieval node.\"\"\"\n    return ensemble_agent.process(state)\n\ndef response_writer_node(state: AgentState) -> AgentState:\n    \"\"\"ResponseWriter node for final response generation.\"\"\"\n    return response_writer_agent.process(state)\n\nprint(\"✅ Agent node functions defined\")"

### Cell 9: Agent Node Functions

## Phase 3: LangGraph Workflow Orchestration

---\n## ✅ Phase 2 Complete: Individual Agent Implementations\n\n**Implemented:**\n- ✅ **BM25 Agent**: Keyword-based search with fallback to vector similarity\n- ✅ **ContextualCompression Agent**: Fast semantic retrieval with Cohere reranking (urgent mode support)\n- ✅ **Ensemble Agent**: Multi-method retrieval combining vector, multi-query, and BM25\n- ✅ **Supervisor Agent**: GPT-4o-powered intelligent query routing with sophisticated decision logic\n- ✅ **ResponseWriter Agent**: GPT-4o-powered contextual response generation with production incident awareness\n\n**Key Features:**\n- 🧠 **Strong reasoning foundation** with GPT-4o for complex decisions\n- ⚡ **Production incident support** with urgent mode optimizations\n- 🔄 **Robust error handling** with graceful fallbacks\n- 📊 **Performance tracking** and detailed metadata\n- 🎯 **Intelligent routing** based on query characteristics and user constraints\n\n**Ready for Phase 3:** LangGraph workflow orchestration!"

In [ ]:
# ResponseWriter Agent - Contextual response generation using GPT-4o\n\nclass ResponseWriterAgent:\n    \"\"\"ResponseWriter agent for generating contextual responses using GPT-4o reasoning.\"\"\"\n    \n    def __init__(self, response_writer_llm):\n        self.response_writer_llm = response_writer_llm\n        self.response_prompt = self._create_response_prompt()\n    \n    def _create_response_prompt(self):\n        \"\"\"Create the response generation prompt.\"\"\"\n        return ChatPromptTemplate.from_template(\"\"\"\n        You are a RESPONSE WRITER agent for a JIRA ticket retrieval system. Generate helpful, contextual responses based on retrieved JIRA ticket information.\n        \n        CONTEXT:\n        Query: {query}\n        Production Incident: {production_incident}\n        Retrieval Method Used: {retrieval_method}\n        \n        RETRIEVED JIRA TICKETS:\n        {retrieved_contexts}\n        \n        INSTRUCTIONS:\n        1. Analyze the user's query and the retrieved JIRA ticket information\n        2. Generate a helpful response that addresses the user's specific question\n        3. If this is a production incident, prioritize urgent/actionable information\n        4. Reference specific JIRA tickets when relevant (use ticket keys like HBASE-123)\n        5. If no relevant information is found, clearly state this\n        6. Keep the response concise but informative\n        \n        RESPONSE STYLE:\n        - Production Incident: Direct, actionable, prioritize immediate solutions\n        - General Query: Comprehensive, educational, include background context\n        - No Results: Suggest alternative search terms or approaches\n        \n        Generate a response that directly answers the user's query:\n        \"\"\")\n    \n    def generate_response(self, query: str, retrieved_contexts: List[Dict], \n                         production_incident: bool, retrieval_method: str) -> str:\n        \"\"\"Generate contextual response based on retrieved information.\"\"\"\n        try:\n            # Format retrieved contexts for the prompt\n            context_text = format_context_for_llm(retrieved_contexts)\n            \n            # Create response chain\n            response_chain = self.response_prompt | self.response_writer_llm | StrOutputParser()\n            \n            # Generate response\n            response = response_chain.invoke({\n                \"query\": query,\n                \"production_incident\": production_incident,\n                \"retrieval_method\": retrieval_method,\n                \"retrieved_contexts\": context_text if context_text != \"No relevant context found.\" else \"No relevant JIRA tickets found for this query.\"\n            })\n            \n            return response.strip()\n            \n        except Exception as e:\n            print(f\"❌ Response generation error: {e}\")\n            \n            # Fallback response\n            if production_incident:\n                return f\"Unable to generate response for production incident query: '{query}'. Please check system logs or contact support immediately.\"\n            else:\n                return f\"Unable to generate response for query: '{query}'. Please try rephrasing your question or contact support.\"\n    \n    def process(self, state: AgentState) -> AgentState:\n        \"\"\"Process state and generate final response.\"\"\"\n        start_time = datetime.now()\n        \n        query = state['query']\n        retrieved_contexts = state.get('retrieved_contexts', [])\n        production_incident = state['production_incident']\n        retrieval_method = state.get('retrieval_method', 'Unknown')\n        \n        incident_label = \"[PRODUCTION INCIDENT]\" if production_incident else \"\"\n        print(f\"✍️  ResponseWriter Agent {incident_label} generating response...\")\n        \n        # Generate response\n        final_answer = self.generate_response(\n            query, retrieved_contexts, production_incident, retrieval_method\n        )\n        \n        # Extract relevant tickets\n        relevant_tickets = extract_ticket_info(retrieved_contexts)\n        \n        # Update state\n        state['final_answer'] = final_answer\n        state['relevant_tickets'] = relevant_tickets\n        \n        # Add processing message\n        state['messages'].append(AIMessage(\n            content=f\"ResponseWriter generated final answer with {len(relevant_tickets)} relevant tickets\"\n        ))\n        \n        processing_time = measure_performance(start_time)\n        print(f\"✅ ResponseWriter completed in {processing_time:.2f}s\")\n        print(f\"   Generated response: {len(final_answer)} characters\")\n        print(f\"   Relevant tickets: {len(relevant_tickets)}\")\n        \n        return state\n\n# Initialize ResponseWriter Agent\nresponse_writer_agent = ResponseWriterAgent(response_writer_llm)\nprint(\"✅ ResponseWriter Agent initialized with GPT-4o reasoning\")"

### Cell 8: ResponseWriter Agent - Contextual Response Generation

In [ ]:
# Supervisor Agent - Intelligent query routing using GPT-4o\n\nclass SupervisorAgent:\n    \"\"\"Supervisor agent for intelligent query routing using GPT-4o reasoning.\"\"\"\n    \n    def __init__(self, supervisor_llm):\n        self.supervisor_llm = supervisor_llm\n        self.routing_prompt = self._create_routing_prompt()\n    \n    def _create_routing_prompt(self):\n        \"\"\"Create the routing decision prompt.\"\"\"\n        return ChatPromptTemplate.from_template(\"\"\"\n        You are a SUPERVISOR agent for a JIRA ticket retrieval system. Your job is to analyze user queries and route them to the most appropriate retrieval agent.\n        \n        AVAILABLE AGENTS:\n        1. BM25 - Fast keyword-based search, best for:\n           - Specific ticket references (e.g., \"HBASE-123\", \"ticket SPR-456\")\n           - Exact error messages or specific terms\n           - Technical acronyms or specific component names\n        \n        2. ContextualCompression - Fast semantic search with reranking, best for:\n           - Production incidents (when speed is critical)\n           - General troubleshooting questions\n           - When user cannot wait long\n        \n        3. Ensemble - Comprehensive multi-method search, best for:\n           - Complex queries requiring thorough analysis\n           - When user can wait for comprehensive results\n           - Research-type questions needing broad coverage\n        \n        ROUTING RULES:\n        - If query contains specific ticket references → BM25\n        - If user_can_wait=True → Ensemble\n        - If production_incident=True (urgent) → ContextualCompression\n        - Default → ContextualCompression\n        \n        QUERY: {query}\n        USER_CAN_WAIT: {user_can_wait}\n        PRODUCTION_INCIDENT: {production_incident}\n        \n        Analyze the query and respond with ONLY:\n        {{\"agent\": \"BM25|ContextualCompression|Ensemble\", \"reasoning\": \"brief explanation\"}}\n        \"\"\")\n    \n    def route_query(self, query: str, user_can_wait: bool, production_incident: bool) -> Dict[str, str]:\n        \"\"\"Route query to appropriate agent.\"\"\"\n        try:\n            # Format prompt\n            routing_chain = self.routing_prompt | self.supervisor_llm | StrOutputParser()\n            \n            # Get routing decision\n            response = routing_chain.invoke({\n                \"query\": query,\n                \"user_can_wait\": user_can_wait,\n                \"production_incident\": production_incident\n            })\n            \n            # Parse JSON response\n            import json\n            try:\n                routing_decision = json.loads(response)\n                agent = routing_decision.get(\"agent\", \"ContextualCompression\")\n                reasoning = routing_decision.get(\"reasoning\", \"Default routing\")\n            except json.JSONDecodeError:\n                # Fallback parsing if JSON is malformed\n                if \"BM25\" in response:\n                    agent = \"BM25\"\n                elif \"Ensemble\" in response:\n                    agent = \"Ensemble\"\n                else:\n                    agent = \"ContextualCompression\"\n                reasoning = \"Parsed from text response\"\n            \n            # Validate agent choice\n            valid_agents = [\"BM25\", \"ContextualCompression\", \"Ensemble\"]\n            if agent not in valid_agents:\n                agent = \"ContextualCompression\"\n                reasoning = \"Invalid agent, using default\"\n            \n            return {\"agent\": agent, \"reasoning\": reasoning}\n            \n        except Exception as e:\n            print(f\"⚠️  Routing error: {e}\")\n            # Safe fallback\n            if production_incident:\n                return {\"agent\": \"ContextualCompression\", \"reasoning\": \"Emergency fallback for production incident\"}\n            elif user_can_wait:\n                return {\"agent\": \"Ensemble\", \"reasoning\": \"Fallback for comprehensive search\"}\n            else:\n                return {\"agent\": \"ContextualCompression\", \"reasoning\": \"Safe default fallback\"}\n    \n    def process(self, state: AgentState) -> AgentState:\n        \"\"\"Process query and determine routing.\"\"\"\n        start_time = datetime.now()\n        \n        query = state['query']\n        user_can_wait = state['user_can_wait']\n        production_incident = state['production_incident']\n        \n        print(f\"🧠 Supervisor Agent analyzing query: '{query}'\")\n        print(f\"   user_can_wait: {user_can_wait}, production_incident: {production_incident}\")\n        \n        # Make routing decision\n        routing_result = self.route_query(query, user_can_wait, production_incident)\n        \n        # Update state\n        state['routing_decision'] = routing_result['agent']\n        state['routing_reasoning'] = routing_result['reasoning']\n        \n        # Add processing message\n        state['messages'].append(AIMessage(\n            content=f\"Supervisor routed query to {routing_result['agent']} agent: {routing_result['reasoning']}\"\n        ))\n        \n        print(f\"✅ Supervisor decision: {routing_result['agent']} - {routing_result['reasoning']}\")\n        print(f\"   Analysis time: {measure_performance(start_time):.2f}s\")\n        \n        return state\n\n# Initialize Supervisor Agent\nsupervisor_agent = SupervisorAgent(supervisor_llm)\nprint(\"✅ Supervisor Agent initialized with GPT-4o reasoning\")"

### Cell 7: Supervisor Agent - Intelligent Query Routing

In [ ]:
# Ensemble Agent - Comprehensive retrieval combining multiple methods\n\nclass EnsembleAgent:\n    \"\"\"Agent for comprehensive retrieval using ensemble of multiple methods.\"\"\"\n    \n    def __init__(self, vectorstore, rag_llm, bm25_agent, contextual_compression_agent, k=10):\n        self.vectorstore = vectorstore\n        self.rag_llm = rag_llm\n        self.bm25_agent = bm25_agent\n        self.contextual_compression_agent = contextual_compression_agent\n        self.k = k\n        self.ensemble_retriever = None\n        self._setup_ensemble_retriever()\n    \n    def _setup_ensemble_retriever(self):\n        \"\"\"Setup ensemble retriever combining multiple methods.\"\"\"\n        try:\n            # Base vector retriever\n            vector_retriever = self.vectorstore.as_retriever(search_kwargs={\"k\": self.k})\n            \n            # Multi-query retriever for query expansion\n            multi_query_retriever = MultiQueryRetriever.from_llm(\n                retriever=vector_retriever,\n                llm=self.rag_llm\n            )\n            \n            # Collect all available retrievers\n            retrievers = [vector_retriever, multi_query_retriever]\n            weights = [0.4, 0.4]  # Base weights\n            \n            # Add BM25 if available\n            if self.bm25_agent.bm25_retriever:\n                retrievers.append(self.bm25_agent.bm25_retriever)\n                weights.append(0.2)\n            \n            # Normalize weights\n            total_weight = sum(weights)\n            weights = [w / total_weight for w in weights]\n            \n            # Create ensemble retriever\n            if len(retrievers) > 1:\n                self.ensemble_retriever = EnsembleRetriever(\n                    retrievers=retrievers,\n                    weights=weights\n                )\n                print(f\"✅ Ensemble retriever initialized with {len(retrievers)} methods\")\n                print(f\"   Methods: Vector ({weights[0]:.2f}), Multi-Query ({weights[1]:.2f})\", end=\"\")\n                if len(weights) > 2:\n                    print(f\", BM25 ({weights[2]:.2f})\")\n                else:\n                    print()\n            else:\n                # Fallback to single retriever\n                self.ensemble_retriever = vector_retriever\n                print(\"✅ Fallback to single vector retriever\")\n                \n        except Exception as e:\n            print(f\"⚠️  Error setting up Ensemble: {e}\")\n            # Fallback to basic retriever\n            self.ensemble_retriever = self.vectorstore.as_retriever(search_kwargs={\"k\": self.k})\n            print(\"✅ Fallback to basic vector retriever\")\n    \n    def retrieve(self, query: str) -> List[Dict[str, Any]]:\n        \"\"\"Perform ensemble retrieval combining multiple methods.\"\"\"\n        try:\n            # Perform ensemble retrieval\n            docs = self.ensemble_retriever.get_relevant_documents(query)\n            \n            # Convert to standardized format\n            results = []\n            seen_content = set()  # Deduplicate similar results\n            \n            for doc in docs:\n                # Simple deduplication based on content hash\n                content_hash = hash(doc.page_content[:200])  # Use first 200 chars\n                if content_hash not in seen_content:\n                    results.append({\n                        'content': doc.page_content,\n                        'metadata': doc.metadata,\n                        'source': 'ensemble',\n                        'score': getattr(doc, 'score', 0.8)\n                    })\n                    seen_content.add(content_hash)\n            \n            return results[:self.k]  # Limit to requested number\n            \n        except Exception as e:\n            print(f\"❌ Ensemble retrieval error: {e}\")\n            return []\n    \n    def process(self, state: AgentState) -> AgentState:\n        \"\"\"Process query using Ensemble agent.\"\"\"\n        start_time = datetime.now()\n        \n        print(f\"🔗 Ensemble Agent processing: '{state['query']}'\")\n        print(\"   Using comprehensive multi-method retrieval...\")\n        \n        # Perform retrieval\n        retrieved_contexts = self.retrieve(state['query'])\n        \n        # Update state\n        state['retrieved_contexts'] = retrieved_contexts\n        state['retrieval_method'] = 'Ensemble'\n        state['retrieval_metadata'] = {\n            'agent': 'Ensemble',\n            'num_results': len(retrieved_contexts),\n            'processing_time': measure_performance(start_time),\n            'method_type': 'multi_method_ensemble',\n            'methods_used': ['vector', 'multi_query', 'bm25'] if self.bm25_agent.bm25_retriever else ['vector', 'multi_query']\n        }\n        \n        # Add processing message\n        state['messages'].append(AIMessage(\n            content=f\"Ensemble Agent retrieved {len(retrieved_contexts)} documents using comprehensive multi-method approach\"\n        ))\n        \n        print(f\"✅ Ensemble Agent completed: {len(retrieved_contexts)} results in {measure_performance(start_time):.2f}s\")\n        return state\n\n# Initialize Ensemble Agent (requires BM25 and ContextualCompression agents)\nensemble_agent = EnsembleAgent(vectorstore, rag_llm, bm25_agent, contextual_compression_agent)\nprint(\"✅ Ensemble Agent initialized\")"

### Cell 6: Ensemble Agent - Comprehensive Multi-Method Retrieval

In [ ]:
# ContextualCompression Agent - Fast semantic retrieval with reranking\n\nclass ContextualCompressionAgent:\n    \"\"\"Agent for fast semantic retrieval with contextual compression and reranking.\"\"\"\n    \n    def __init__(self, vectorstore, rag_llm, k=10):\n        self.vectorstore = vectorstore\n        self.rag_llm = rag_llm\n        self.k = k\n        self.compression_retriever = None\n        self._setup_compression_retriever()\n    \n    def _setup_compression_retriever(self):\n        \"\"\"Setup contextual compression retriever with Cohere reranking.\"\"\"\n        try:\n            # Base retriever from vectorstore\n            base_retriever = self.vectorstore.as_retriever(search_kwargs={\"k\": self.k * 2})  # Get more for reranking\n            \n            # Try to setup Cohere reranking\n            try:\n                compressor = CohereRerank(model=\"rerank-v3.5\")\n                self.compression_retriever = ContextualCompressionRetriever(\n                    base_compressor=compressor,\n                    base_retriever=base_retriever\n                )\n                print(\"✅ ContextualCompression with Cohere reranking initialized\")\n                \n            except Exception as cohere_error:\n                print(f\"⚠️  Cohere reranking unavailable: {cohere_error}\")\n                print(\"🔄 Using LLM-based contextual compression instead\")\n                \n                from langchain.retrievers.document_compressors import LLMChainExtractor\n                \n                # Fallback to LLM-based compression\n                compressor = LLMChainExtractor.from_llm(self.rag_llm)\n                self.compression_retriever = ContextualCompressionRetriever(\n                    base_compressor=compressor,\n                    base_retriever=base_retriever\n                )\n                print(\"✅ ContextualCompression with LLM compression initialized\")\n                \n        except Exception as e:\n            print(f\"⚠️  Error setting up ContextualCompression: {e}\")\n            # Fallback to basic retriever\n            self.compression_retriever = self.vectorstore.as_retriever(search_kwargs={\"k\": self.k})\n            print(\"✅ Fallback to basic vector retriever\")\n    \n    def retrieve(self, query: str, is_urgent: bool = False) -> List[Dict[str, Any]]:\n        \"\"\"Perform contextual compression retrieval.\"\"\"\n        try:\n            # Adjust parameters for urgent queries\n            if is_urgent:\n                # For production incidents, prioritize speed\n                search_kwargs = {\"k\": min(self.k, 5)}  # Fewer results for speed\n            else:\n                search_kwargs = {\"k\": self.k}\n            \n            # Perform retrieval\n            if hasattr(self.compression_retriever, 'get_relevant_documents'):\n                docs = self.compression_retriever.get_relevant_documents(query)\n            else:\n                docs = self.compression_retriever.invoke(query)\n            \n            # Convert to standardized format\n            results = []\n            for doc in docs:\n                results.append({\n                    'content': doc.page_content,\n                    'metadata': doc.metadata,\n                    'source': 'contextual_compression',\n                    'score': getattr(doc, 'score', 0.9)\n                })\n            \n            return results[:search_kwargs[\"k\"]]  # Limit results\n            \n        except Exception as e:\n            print(f\"❌ ContextualCompression retrieval error: {e}\")\n            return []\n    \n    def process(self, state: AgentState) -> AgentState:\n        \"\"\"Process query using ContextualCompression agent.\"\"\"\n        start_time = datetime.now()\n        \n        is_urgent = state.get('production_incident', False)\n        urgency_label = \"[URGENT]\" if is_urgent else \"\"\n        \n        print(f\"⚡ ContextualCompression Agent {urgency_label} processing: '{state['query']}'\")\n        \n        # Perform retrieval with urgency consideration\n        retrieved_contexts = self.retrieve(state['query'], is_urgent=is_urgent)\n        \n        # Update state\n        state['retrieved_contexts'] = retrieved_contexts\n        state['retrieval_method'] = 'ContextualCompression'\n        state['retrieval_metadata'] = {\n            'agent': 'ContextualCompression',\n            'num_results': len(retrieved_contexts),\n            'processing_time': measure_performance(start_time),\n            'method_type': 'semantic_with_reranking',\n            'is_urgent': is_urgent\n        }\n        \n        # Add processing message\n        urgency_note = \" (urgent mode)\" if is_urgent else \"\"\n        state['messages'].append(AIMessage(\n            content=f\"ContextualCompression Agent retrieved {len(retrieved_contexts)} documents using semantic search with reranking{urgency_note}\"\n        ))\n        \n        print(f\"✅ ContextualCompression Agent completed: {len(retrieved_contexts)} results in {measure_performance(start_time):.2f}s\")\n        return state\n\n# Initialize ContextualCompression Agent\ncontextual_compression_agent = ContextualCompressionAgent(vectorstore, rag_llm)\nprint(\"✅ ContextualCompression Agent initialized\")"

### Cell 5: ContextualCompression Agent - Fast Semantic Retrieval

In [ ]:
# BM25 Agent - Keyword-based retrieval for specific ticket searches\nfrom datetime import datetime\n\nclass BM25Agent:\n    \"\"\"Agent for keyword-based search using BM25 algorithm.\"\"\"\n    \n    def __init__(self, vectorstore, rag_llm, k=10):\n        self.vectorstore = vectorstore\n        self.rag_llm = rag_llm\n        self.k = k\n        self.bm25_retriever = None\n        self._setup_bm25_retriever()\n    \n    def _setup_bm25_retriever(self):\n        \"\"\"Setup BM25 retriever from vectorstore documents.\"\"\"\n        try:\n            # Get documents from vectorstore for BM25 indexing\n            if hasattr(self.vectorstore, 'similarity_search'):\n                # Get sample documents to build BM25 index\n                sample_docs = self.vectorstore.similarity_search(\n                    \"sample query\", k=100  # Get more docs for better BM25 performance\n                )\n                \n                if sample_docs:\n                    self.bm25_retriever = BM25Retriever.from_documents(\n                        sample_docs, k=self.k\n                    )\n                    print(f\"✅ BM25 retriever initialized with {len(sample_docs)} documents\")\n                else:\n                    print(\"⚠️  No documents found for BM25 indexing\")\n                    self.bm25_retriever = None\n            else:\n                print(\"⚠️  Vectorstore doesn't support similarity search\")\n                self.bm25_retriever = None\n                \n        except Exception as e:\n            print(f\"⚠️  Error setting up BM25 retriever: {e}\")\n            self.bm25_retriever = None\n    \n    def retrieve(self, query: str) -> List[Dict[str, Any]]:\n        \"\"\"Perform BM25-based retrieval.\"\"\"\n        try:\n            if self.bm25_retriever:\n                # Use BM25 retriever\n                docs = self.bm25_retriever.get_relevant_documents(query)\n            else:\n                # Fallback to vectorstore similarity search\n                docs = self.vectorstore.similarity_search(query, k=self.k)\n            \n            # Convert to standardized format\n            results = []\n            for doc in docs:\n                results.append({\n                    'content': doc.page_content,\n                    'metadata': doc.metadata,\n                    'source': 'bm25',\n                    'score': getattr(doc, 'score', 1.0)\n                })\n            \n            return results\n            \n        except Exception as e:\n            print(f\"❌ BM25 retrieval error: {e}\")\n            return []\n    \n    def process(self, state: AgentState) -> AgentState:\n        \"\"\"Process query using BM25 agent.\"\"\"\n        start_time = datetime.now()\n        \n        print(f\"🔍 BM25 Agent processing: '{state['query']}'\")\n        \n        # Perform retrieval\n        retrieved_contexts = self.retrieve(state['query'])\n        \n        # Update state\n        state['retrieved_contexts'] = retrieved_contexts\n        state['retrieval_method'] = 'BM25'\n        state['retrieval_metadata'] = {\n            'agent': 'BM25',\n            'num_results': len(retrieved_contexts),\n            'processing_time': measure_performance(start_time),\n            'method_type': 'keyword_based'\n        }\n        \n        # Add processing message\n        state['messages'].append(AIMessage(\n            content=f\"BM25 Agent retrieved {len(retrieved_contexts)} documents using keyword search\"\n        ))\n        \n        print(f\"✅ BM25 Agent completed: {len(retrieved_contexts)} results in {measure_performance(start_time):.2f}s\")\n        return state\n\n# Initialize BM25 Agent\nbm25_agent = BM25Agent(vectorstore, rag_llm)\nprint(\"✅ BM25 Agent initialized\")"

### Cell 4: BM25 Agent - Keyword-Based Search

## Phase 2: Individual Agent Implementations

### Cell 3: State Schema & Shared Components

In [ ]:
# Define the state schema for the multi-agent system
class AgentState(TypedDict):
    """State shared between all agents in the graph."""
    
    # Input parameters
    query: str
    user_can_wait: bool
    production_incident: bool
    
    # Routing decisions
    routing_decision: Optional[str]  # Which agent to use
    routing_reasoning: Optional[str]  # Why this agent was chosen
    
    # Retrieval results
    retrieved_contexts: List[Dict[str, Any]]
    retrieval_method: Optional[str]  # Which method was used
    retrieval_metadata: Dict[str, Any]  # Performance metrics, etc.
    
    # Final response
    final_answer: Optional[str]
    relevant_tickets: List[Dict[str, str]]  # key, title pairs
    
    # System metadata
    messages: Annotated[List[BaseMessage], add_messages]
    timestamp: str
    processing_time: Optional[float]

print("✅ State schema defined")

In [ ]:
# Shared utility functions

def format_context_for_llm(retrieved_contexts: List[Dict]) -> str:
    """Format retrieved contexts for LLM consumption."""
    if not retrieved_contexts:
        return "No relevant context found."
    
    context_parts = []
    for i, ctx in enumerate(retrieved_contexts[:10]):  # Limit to top 10
        content = ctx.get('content', '')
        metadata = ctx.get('metadata', {})
        key = metadata.get('key', f'DOC-{i+1}')
        
        context_parts.append(f"[{key}] {content}")
    
    return "\n\n---\n\n".join(context_parts)

def extract_ticket_info(retrieved_contexts: List[Dict]) -> List[Dict[str, str]]:
    """Extract ticket key and title information."""
    tickets = []
    seen_keys = set()
    
    for ctx in retrieved_contexts:
        metadata = ctx.get('metadata', {})
        key = metadata.get('key', '')
        
        if key and key not in seen_keys:
            # Extract title from content or metadata
            title = metadata.get('title', '')
            if not title:
                # Try to extract from content
                content = ctx.get('content', '')
                if content.startswith('Title: '):
                    title = content.split('\n')[0].replace('Title: ', '')
                else:
                    title = content[:100] + '...' if len(content) > 100 else content
            
            tickets.append({
                'key': key,
                'title': title
            })
            seen_keys.add(key)
    
    return tickets

def measure_performance(start_time: datetime) -> float:
    """Calculate processing time in seconds."""
    return (datetime.now() - start_time).total_seconds()

print("✅ Utility functions defined")

---
## ✅ Phase 1 Complete: Setup & Infrastructure

**Implemented:**
- ✅ Dependencies and configuration
- ✅ API keys and LangSmith tracing setup
- ✅ GPT-4o models for reasoning (Supervisor + ResponseWriter)
- ✅ GPT-4o-mini for task execution (RAG agents)
- ✅ Qdrant connection with fallback to sample data
- ✅ Shared state schema for agent communication
- ✅ Utility functions for context formatting

**Ready for Phase 2:** Individual agent implementations with strong reasoning foundation!

The system is configured to use GPT-4o for complex reasoning tasks, ensuring robust decision-making for current and future agents.